In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import numpy as np
from hmmlearn import hmm
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Read lane-change data
lc_dict = pd.read_pickle('data/lane_change_dict.pickle')

In [ ]:
# Set lane_pair_list to plot
lane_pair_list = [(1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 5), (5, 4), (4, 3), (3, 2), (2, 1)]

In [ ]:
# Extact lateral displacements and velocities
x_seq_list = list()
v_seq_list = list()
lc_time_list = list()

for key in lane_pair_list:
    flip = 1.0
    if key[0] > key[1]:
        flip = -1.0
    
    lc_df_list = lc_dict[key]
    for ll in lc_df_list:
        x_seq = np.atleast_2d(ll.local_x_normalized.tolist()).T
        v_seq = np.atleast_2d(ll.vel_x.tolist()).T
        if (not np.isnan(x_seq).any()) and (not np.isnan(v_seq).any()):
            lc_time_list.append(0.1*ll.lc_frame_index)
            x_seq_list.append(x_seq)
            v_seq_list.append(flip*v_seq)

In [ ]:
# Stack the observations
obs_seq_list = list()
for i in range(len(x_seq_list)):
    obs = np.hstack([x_seq_list[i],v_seq_list[i]])
    obs_seq_list.append(obs)

In [ ]:
# Save the observation sequence list
# with open('obs_seq_list_lanechange.pickle','wb') as f:
#     pickle.dump(obs_seq_list, f)

In [ ]:
# Load from pickle file
# obs_seq_list = pd.read_pickle('obs_seq_list_lanechange.pickle')

In [ ]:
# Create GaussianHMM model
# model_fit = hmm.GMMHMM(n_components=3,n_mix=1,n_iter=1000)
model_fit = hmm.GaussianHMM(n_components=4, n_iter=100, covariance_type='full')

In [ ]:
# Extract and convert training set
obs_train_list = obs_seq_list[:500]
seq_length_list = list()
num_of_sample = 0
for obs in obs_train_list:
    seq_length = obs.shape[0]
    num_of_sample = seq_length + num_of_sample
    seq_length_list.append(seq_length)
obs_array = np.vstack(obs_train_list)

In [ ]:
# Train the GHMM
model_fit.fit(obs_array,seq_length_list)

In [ ]:
with open('data/hmm_lc_4.pickle', 'wb') as handle:
  pickle.dump(model_fit, handle)

In [ ]:
# Plot emission distribution
fig = plt.figure(figsize=(8,6))
colorlist = 'rgbcym'
for g_index in range(len(model_fit.means_)):
    # Sample each emission distribution for visualization     
    gmm = hmm.GMM(n_components=1,covariance_type='full')
    gmm.weights_ = np.array([1])
    gmm.means_ = np.array([model_fit.means_[g_index]])
    gmm.covars_ = np.array([model_fit.covars_[g_index]])
    gmm_sample = gmm.sample(2000)
    plt.plot(gmm_sample[:,0],gmm_sample[:,1],'%so' %(colorlist[g_index]),alpha=0.2)
    mean_point = model_fit.means_[g_index] 
    plt.text(mean_point[0],mean_point[1],'%s' %(g_index),size=17, bbox=dict(alpha=1.0, facecolor='w'))
    
plt.xlabel('Normalized Lateral Displacement')
plt.ylabel('Lateral velocity')

In [ ]:
# Transition matrix
model_fit.transmat_

In [ ]:
# Evaluate most likely transition trajectory and liklihood
obs_seq = obs_seq_list[1050]
plt.plot(obs_seq[:,0],obs_seq[:,1],'-k',linewidth=3)
log_likelyhood, state_seq = model_fit.decode(obs_seq)
print "State transition sequence: %s" %(state_seq)
print "Log probability of state trajectory: %s" %(log_likelyhood)
print "Score under model: %s" %(model_fit.score(obs_seq))
print "Score under model and prior: %s" %(model_fit.score_samples(obs_seq)[0])

for index, point in enumerate(obs_seq):
    state_index = state_seq[index]
    plt.plot(point[0],point[1],'%so' %(colorlist[state_index]), markeredgewidth=3,markersize=10)

# Train Lane-keeping HMM #

In [ ]:
# Train hmm for lane-keeping
# lk_obs_list = pd.read_pickle('data/lane_keeping_obs_seq_list.pickle')
lk_obs_list = pd.read_pickle('data/obs_seq_list_lanekeep_new.pickle')

In [ ]:
model_fit_lk = hmm.GaussianHMM(n_components=2, n_iter=100, covariance_type='full')

In [ ]:
# Extract and convert training set
obs_train_list = lk_obs_list[:500]
seq_length_list = list()
num_of_sample = 0
for obs in obs_train_list:
    seq_length = obs.shape[0]
    num_of_sample = seq_length + num_of_sample
    seq_length_list.append(seq_length)
obs_array = np.vstack(obs_train_list)

In [ ]:
model_fit_lk.fit(obs_array,seq_length_list)

In [ ]:
# Plot emission distribution
fig = plt.figure(figsize=(8,6))
colorlist = 'rgbcym'
for g_index in range(len(model_fit_lk.means_)):
    # Sample each emission distribution for visualization     
    gmm = hmm.GMM(n_components=1,covariance_type='full')
    gmm.weights_ = np.array([1])
    gmm.means_ = np.array([model_fit_lk.means_[g_index]])
    gmm.covars_ = np.array([model_fit_lk.covars_[g_index]])
    gmm_sample = gmm.sample(2000)
    plt.plot(gmm_sample[:,0],gmm_sample[:,1],'%so' %(colorlist[g_index]),alpha=0.2)
    mean_point = model_fit_lk.means_[g_index] 
    plt.text(mean_point[0],mean_point[1],'%s' %(g_index),size=17, bbox=dict(alpha=1.0, facecolor='w'))
    
plt.xlabel('Normalized Lateral Displacement')
plt.ylabel('Lateral velocity')

In [ ]:
with open('data/hmm_lk_2.pickle', 'wb') as handle:
  pickle.dump(model_fit_lk, handle)